# Group

1. Abdullahil Kafi
2. Israt Nowshin
3. Md Zahangir Hossain



In [1]:
import tensorflow as tf
import numpy as np
import keras
import keras as k

import collections
import matplotlib.pyplot as plt


from tensorflow.keras import layers



Using TensorFlow backend.


In [2]:
model = tf.keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=70, output_dim=70))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(512))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 70)          4900      
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1193984   
_________________________________________________________________
dense (Dense)                (None, 10)                5130      
Total params: 1,204,014
Trainable params: 1,204,014
Non-trainable params: 0
_________________________________________________________________


In [5]:
!python prepare_data2.py shakespeare_input.txt skp \n -m 500

2020-06-03 13:51:24.523566: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 200681 sequences...
Longest sequence is 335 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 500 characters...
200681 sequences remaining.
Longest remaining sequence has length 335.
Removing length-0 sequences...
200681 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences

In [6]:
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x))
data = data.shuffle(46000).padded_batch(128).repeat()

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(data)
print(vocab)
print(vocab_size)

<RepeatDataset shapes: (None, None), types: tf.int32>
{'W': 3, 'L': 4, '.': 5, 'l': 6, 't': 7, 'r': 8, 'C': 9, 'h': 10, 'k': 11, '\n': 12, 'x': 13, 'j': 14, 'V': 15, 'w': 16, 'H': 17, 'J': 18, '?': 19, 'g': 20, 'M': 21, 'i': 22, '[': 23, 'F': 24, 'd': 25, 'c': 26, 'f': 27, '-': 28, '&': 29, ';': 30, 'q': 31, 'u': 32, 'A': 33, '3': 34, ',': 35, 'P': 36, '$': 37, 'Q': 38, ':': 39, ']': 40, 'R': 41, 'I': 42, 'a': 43, ' ': 44, 'T': 45, 'G': 46, 'y': 47, 'p': 48, 'v': 49, 'o': 50, 'S': 51, 'm': 52, 'N': 53, 'X': 54, 'e': 55, 'O': 56, 'b': 57, 'K': 58, 'Z': 59, 'E': 60, "'": 61, '!': 62, 'z': 63, 's': 64, 'B': 65, 'D': 66, 'Y': 67, 'n': 68, 'U': 69, '<PAD>': 0, '<S>': 1, '</S>': 2}
70


In [7]:
for ind, thing in enumerate(data):
    inds = thing[0].numpy()
    to_chars = "".join([ind_to_ch[ind] for ind in inds])
    #print(ind)
    print(to_chars)
    #print()
    break

<S> charge
That </S><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>


In [0]:
count = 0
s = []
for i in data:
  count+=1
  if count == 10:
    print(i)
    getChar(i)


  

In [0]:
#input = tf.one_hot(enumerate(data), vocab_size)

input_dim = 70
hidden_dim = 512  # size of hidden layer of neurons
output_dim = 70
batch = 128
seq_length = 200 # number of steps to unroll the RNN for
learning_rate = 0.001
wt = 0.1




# model parameters
Wxh = tf.Variable(tf.random.uniform([input_dim, hidden_dim], minval = -wt, maxval =wt), dtype=np.float32) #input to hidden
Whh = tf.Variable(tf.random.uniform([hidden_dim, hidden_dim], minval = -wt, maxval =wt), dtype=np.float32) #hidden to hidden
Why = tf.Variable(tf.random.uniform([hidden_dim, output_dim], minval = -wt, maxval =wt), dtype=np.float32)  # hidden to output
bh = tf.Variable(tf.random.uniform([hidden_dim], minval=-wt, maxval=wt), dtype=np.float32) # hidden bias
by = tf.Variable(tf.random.uniform([output_dim], minval=-wt, maxval=wt), dtype=np.float32) # output bias

#initial h
h_0 = tf.reshape(tf.Variable(np.zeros(hidden_dim), dtype=np.float32), (1, 512))

all_vars = [Wxh, Whh, Why, bh, by]

In [0]:
steps = 20*35000 // 128
opt = tf.optimizers.Adam()
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
@tf.function
def rnn(input_seq):
    with tf.GradientTape() as tape:
      state = tf.zeros([tf.shape(input_seq)[0], hidden_dim])
      total_loss = tf.constant(0.)

      for time_step in tf.range(tf.shape(input_seq)[1] - 1):
          inp_here = tf.one_hot(input_seq[:, time_step], vocab_size)
          state_0 = tf.matmul(inp_here, Wxh) + tf.matmul(state, Whh) + bh
          state = tf.nn.tanh(state_0)
          logits = tf.matmul(state, Why) + by

          loss_here = loss_fn(input_seq[:, time_step+1], logits)
          total_loss += loss_here
            
          total_loss /= tf.cast(tf.shape(input_seq)[1] - 1, tf.float32)
    grads = tape.gradient(total_loss, all_vars)
    
      # this is gradient clipping
    glob_norm = tf.linalg.global_norm(grads)
    grads = [g/glob_norm for g in grads]
    
    opt.apply_gradients(zip(grads, all_vars))

    return total_loss
   



In [9]:
for step, seqs in enumerate(data):
    xent_avg = rnn(seqs)

    if not step % 200:
        print("Step: {} Loss: {}".format(step, xent_avg))
        print()
        

    if step > steps:
        break

Step: 0 Loss: 0.054114922881126404

Step: 200 Loss: 0.0006541435141116381

Step: 400 Loss: 0.00022947405523154885

Step: 600 Loss: 1.7327951354673132e-05

Step: 800 Loss: 7.350729447352933e-06

Step: 1000 Loss: 1.4714621102029923e-05

Step: 1200 Loss: 1.198326231133251e-06

Step: 1400 Loss: 4.174125024292152e-06

Step: 1600 Loss: 0.0005927584716118872

Step: 1800 Loss: 7.800545063219033e-06

Step: 2000 Loss: 4.101102149434155e-06

Step: 2200 Loss: 1.549561147839995e-06

Step: 2400 Loss: 1.7254391423193738e-05

Step: 2600 Loss: 0.002169656567275524

Step: 2800 Loss: 2.2515641830977984e-06

Step: 3000 Loss: 6.787024631194072e-06

Step: 3200 Loss: 0.00017857932834886014

Step: 3400 Loss: 5.2482505452644546e-06

Step: 3600 Loss: 6.964741260162555e-06

Step: 3800 Loss: 1.0876675332838204e-05

Step: 4000 Loss: 5.254797997622518e-06

Step: 4200 Loss: 7.670268132642377e-06

Step: 4400 Loss: 4.7625821935071144e-06

Step: 4600 Loss: 1.737801767376368e-06

Step: 4800 Loss: 5.0152093535871245e-06


In [0]:
 model.fit(enumerate(data))

RuntimeError: ignored

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 70

# Number of RNN units
rnn_units = 512
BATCH_SIZE = 128

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (128, None, 70)           4900      
_________________________________________________________________
gru (GRU)                    (128, None, 512)          897024    
_________________________________________________________________
dense_1 (Dense)              (128, None, 70)           35910     
Total params: 937,834
Trainable params: 937,834
Non-trainable params: 0
_________________________________________________________________


In [14]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

NameError: ignored

In [0]:
for step, seqs in enumerate(data):
  for time_step in tf.range(tf.shape(seqs)[1] - 1):
          inp_here = tf.one_hot(seqs[:, time_step], vocab_size)
          example_batch_predictions = model(inp_here)
          print(example_batch_predictions.shape)
    

Streaming output truncated to the last 5000 lines.
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 70)
(128, 70, 7

KeyboardInterrupt: ignored

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([44, 26, 43, 26,  4, 56,  7,  3, 52,  2, 61, 63, 35, 51, 50, 12, 49,
        6, 63, 36, 48, 44,  9, 37,  0, 64,  2, 63, 31,  3, 60, 60, 10, 48,
        6, 55, 24, 64, 65, 26, 23, 34, 41, 54, 68, 42, 11, 30, 32, 29, 49,
       58, 61, 60, 51, 19, 36, 45, 33,  8, 40, 54,  6, 12, 61,  4,  8, 60,
       64, 48])

In [0]:

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)



In [0]:

print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

IndexError: ignored

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

ValueError: ignored